# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [6]:
data 

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu Aug 18 16:51:12 2016',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.23001961,  5.0725783 , -0.27606055,  0.83244412, -0.37786573,
          0.4803223 ],
        [ 0.15549112, -0.16939038,  0.67065219, -0.85955255, -0.37786573,
         -0.94572324],
        [-0.78441482, -0.44365372,  5.6747053 , -0.85955255, -0.37786573,
         -0.94572324],
        ...,
        [ 1.2049878 ,  1.7637238 , -0.50146835,  1.5624078 ,  6.4890725 ,
          0.93129397],
        [ 0.73664398, -0.22247361, -0.05065276,  1.5096647 ,  0.53926914,
          1.3152293 ],
        [ 0.17700275, -0.19150839, -0.50146835,  1.5788636 ,  7.750705  ,
          1.5559507 ]]),
 'y': array([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=uint8)}

In [55]:
train_split=6709
test_split=6709+1677

np.shape(X[:train_split])
np.shape(X[train_split:test_split])
np.shape(X[test_split:])

np.shape(Y[:train_split])
np.shape(Y[train_split:test_split])
np.shape(Y[test_split:])


(2797, 6)

In [54]:
size=np.shape(X)[0]
import math
np.shape(X[:math.floor(size*0.6)])
np.shape(X[math.floor(size*0.6):math.floor(size*0.75)])
np.shape(X[math.floor(size*0.75):])

(2796, 6)

In [28]:
import pandas as pd
pd_X=pd.DataFrame(X)
pd_Y=pd.DataFrame(Y,columns=['Y'])
pd_X
pd.concat([pd_X,pd_Y], axis=1)

,0,1,2,3,4,5,Y
0,-0.051156,-0.333064,-0.321142,1.488568,-0.377866,0.504699,0
1,0.122800,-0.372876,0.084592,1.021897,0.921170,1.354842,1
2,-0.784415,-0.470195,-0.591631,-0.859553,-0.377866,-0.945723,0
3,-0.784415,-0.470195,-0.591631,-0.859553,-0.377866,-0.945723,0
4,0.404315,0.702059,-0.411305,0.688139,-0.377866,1.101932,0
...,...,...,...,...,...,...,...
11178,-0.136356,-0.421536,-0.095734,0.766199,2.308445,0.986142,0
11179,-0.097059,-0.372876,0.129673,-0.859553,-0.377866,-0.945723,0
11180,-0.784415,-0.470195,-0.591631,-0.859553,-0.377866,-0.945723,0
11181,-0.784415,-0.470195,-0.591631,-0.859553,-0.377866,-0.945723,0


In [59]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline   
from sklearn.linear_model import LogisticRegression

# 將先前處理的 Transformer 與模型 Estimator 同樣透過一序列的 tuple 建置成 Pipeline
model= Pipeline(steps=[('scaler', StandardScaler()),
                         ('classifier', LogisticRegression())])
model.fit(X_train, Y_train)
# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)



train_split=6709
test_split=6709+1677

X_train=X[:train_split]
X_test=X[train_split:test_split]
X_valid=X[test_split:]

Y_train=Y[:train_split]
Y_test=Y[train_split:test_split]
Y_valid=Y[test_split:]


[np.shape(X[:train_split]),np.shape(X[train_split:test_split]),np.shape(X[test_split:]),np.shape(Y[:train_split]),np.shape(Y[train_split:test_split]),np.shape(Y[test_split:])]

[(6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)]

In [ ]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline   
from sklearn.linear_model import LogisticRegression

# 將先前處理的 Transformer 與模型 Estimator 同樣透過一序列的 tuple 建置成 Pipeline
model= Pipeline(steps=[('scaler', StandardScaler()),
                         ('classifier', LogisticRegression())])
model.fit(X_train, Y_train)

In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
print(scaler.mean_)

[ 0.00210728 -0.00049436 -0.00648506 -0.00909968 -0.01076254 -0.0083896 ]


In [95]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline   

from sklearn.linear_model import LogisticRegression

# 將先前處理的 Transformer 與模型 Estimator 同樣透過一序列的 tuple 建置成 Pipeline
model= Pipeline(steps=[('scaler', StandardScaler()),
                         ('classifier', LogisticRegression())])
model.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression())])

In [96]:
np.mean(model.predict(X_train)==Y_train)

0.9830078998360411

In [97]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

y_pred=model.predict(X_test)

intervals_output=classification_report_interval(
    Y_test,
    y_pred,
    alpha = 0.05,
    union_bound_correction=True
)
print(intervals_output)
# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

precision0 = (0.94,1.00)
recall0 = (0.96,1.00)
precision1 = (0.33,1.00)
recall1 = (0.09,0.62)


# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)



            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



In [186]:
# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    p0_t1=np.sum(np.multiply(predictions==0,Y==1))
    p1_t0=np.sum(np.multiply(predictions==1,Y==0))
    
    average_cost=(30*p0_t1+5*p1_t0)/len(Y)
    
    return average_cost

In [181]:
threshold=1
pred_proba =model.predict_proba(X_test)[:,1]
predictions = (pred_proba >= threshold)*1
np.mean(predictions)
from scipy.stats.contingency import crosstab
(labels, _), table = crosstab(predictions==1, Y_test)
table
39*30/len(Y_test)


0.6976744186046512

In [188]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost = cost(model,1,X_test,Y_test)
[naive_cost]

[0.6976744186046512]

In [202]:

# Part 5

cost_list=[]
threshold_candidates=np.linspace(0, 1, num=100)
for threshold in threshold_candidates:
    cost_list.append(cost(model,threshold,X_test,Y_test))

index=cost_list.index(min(cost_list))

optimal_threshold = threshold_candidates[index]
cost_at_optimal_threshold = cost_list[index]

[optimal_threshold,cost_at_optimal_threshold]

[0.11111111111111112, 0.3518187239117472]

In [214]:

# Part 6

cost_at_optimal_threshold_validation = cost(model,optimal_threshold,X_valid,Y_valid)



# Report the cost interval as a tuple cost_interval = (a,b)
#a=0,b=30 
n=len(Y_valid)
a=0
b=30 
alpha=0.05

def compute_epsilon(n,a,b,alpha):
    return np.sqrt((b-a)**2/(2*n)*np.log(2/alpha))
# fill in the calculated l from part 3 here
epsilon = compute_epsilon(n,a,b,alpha)

cost_interval = (cost_at_optimal_threshold_validation-epsilon,cost_at_optimal_threshold_validation+epsilon)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

[cost_at_optimal_threshold_validation,epsilon,cost_interval]

[0.2913836253128352,
 0.7703840018556184,
 (-0.4790003765427832, 1.0617676271684535)]

In [ ]:

# Part 7

variance_of_C = XXX

In [ ]:

# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality
interval_of_C = XXX

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)